In [1]:
using Revise
include("ngrams.jl")
include("datasetloader.jl")
LANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", 
"eng", "epo", "fas", "fin", "fra", "hau", "hbs", "heb", "hin", "hun", 
"ido", "ina", "isl", "ita", "jpn", "kab", "kor", "kur", "lat", "lit", 
"mar", "mkd", "msa", "nds", "nld", "nor", "pol", "por", "ron", "rus", 
"slk", "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", 
"zho"]
G = [lang => load_ngrams(joinpath("ngrams", lang * ".txt")) for lang in LANGS]

50-element Vector{Pair{String, Dict{Tuple{UInt8, Vararg{UInt8}}, Float64}}}:
 "ara" => Dict((0x63, 0x61) => 0.00029467622074500394, (0xd9, 0x8a, 0x20, 0xd9, 0x85) => 0.006367103586697142, (0x20, 0xd8, 0xa7, 0x20) => 0.00540693369022176, (0xa7, 0xd9, 0x86, 0x20, 0xd9) => 0.005268843596716616, (0xd9, 0x8a, 0xd8) => 0.10189825473245379, (0x68, 0x65) => 0.00010555566116238947, (0x63, 0x20, 0x6d) => 0.00010555566116238947, (0xb9, 0x20, 0xd9) => 0.0058931359699533415, (0xd8, 0xb4, 0xd8) => 0.01657449209142566, (0xa7, 0x20, 0xd8, 0xa7, 0xd9) => 0.005374842588265839…)
 "bel" => Dict((0x8c, 0xd0, 0xba) => 0.0028673775998338327, (0xd1, 0x87) => 0.03788978898882851, (0xd0, 0xb3, 0xd0, 0xbb) => 0.0028589731977762345, (0x8c, 0xd1, 0x81, 0xd1) => 0.00013294853270790118, (0x68, 0x65) => 0.0002951312235264679, (0xbe, 0x20, 0xd0, 0xba) => 0.00045075712967128454, (0xb4, 0xd0, 0xb7, 0xd0) => 0.010029318549427318, (0xb3, 0xd0, 0xbd, 0xd0) => 0.00026223826333618465, (0x20, 0xd0, 0xb3, 0xd0, 0xb5) => 0.0010

In [2]:
sort([l=>sum(values(D)) for (l,D) in G], by=last, rev=true)

50-element Vector{Pair{String, Float64}}:
 "rus" => 32.64583283393606
 "mkd" => 30.66520503485124
 "mar" => 29.90013761875782
 "ukr" => 29.87017392828141
 "ben" => 29.19866258235814
 "bul" => 29.064488592888104
 "hin" => 28.8075530768619
 "heb" => 28.11414717451105
 "hbs" => 28.074124203261754
 "bel" => 27.030819344124
       ⋮
 "ces" => 22.09354922569602
 "slk" => 22.07343820462436
 "lit" => 22.04530106959115
 "isl" => 22.029997624397925
 "pol" => 21.3133521252766
 "vie" => 21.28969858889615
 "jpn" => 20.709148162900426
 "kor" => 17.530313346203425
 "zho" => 15.890838106506262

In [3]:
function norm_table!(t)
    D = last(t)
    vs = sum(values(D))
    for (k, v) in D
        D[k] /= vs
        D[k] = log(D[k])
    end
end

norm_table! (generic function with 1 method)

In [4]:
norm_table!.(G);

In [5]:
LOG_T = last.(G)

50-element Vector{Dict{Tuple{UInt8, Vararg{UInt8}}, Float64}}:
 Dict((0x63, 0x61) => -11.323687210192006, (0xd9, 0x8a, 0x20, 0xd9, 0x85) => -8.250664457783948, (0x20, 0xd8, 0xa7, 0x20) => -8.414126981363069, (0xa7, 0xd9, 0x86, 0x20, 0xd9) => -8.439998220884934, (0xd9, 0x8a, 0xd8) => -5.477834315172113, (0x68, 0x65) => -12.350325999235027, (0x63, 0x20, 0x6d) => -12.350325999235027, (0xb9, 0x20, 0xd9) => -8.328020849277252, (0xd8, 0xb4, 0xd8) => -7.293944235445375, (0xa7, 0x20, 0xd8, 0xa7, 0xd9) => -8.420079840315696…)
 Dict((0x8c, 0xd0, 0xba) => -9.15133506742282, (0xd1, 0x87) => -6.570051295240793, (0xd0, 0xb3, 0xd0, 0xbb) => -9.154270412597846, (0x8c, 0xd1, 0x81, 0xd1) => -12.222526148983766, (0x68, 0x65) => -11.425068147243712, (0xbe, 0x20, 0xd0, 0xba) => -11.001559550881396, (0xb4, 0xd0, 0xb7, 0xd0) => -7.899220292777139, (0xb3, 0xd0, 0xbd, 0xd0) => -11.543234737639356, (0x20, 0xd0, 0xb3, 0xd0, 0xb5) => -10.15397480770146, (0x8b, 0xd0, 0xb3) => -9.792912335585456…)
 Dict((0xb0, 0x20

In [6]:
DEFAULT_LOGP = (minimum.(values.(last.((G))))|>minimum)

-16.978494208689185

In [52]:
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)
TV[rand(1:length(TV))]

"¿Quieres que me quede?" => "spa"

In [7]:
function likelihood(t, logt, default_logp=DEFAULT_LOGP)
    sc = 0.
    for (code, p) in t
        sc += p * get(logt, code, default_logp)
    end
    sc
end
    

likelihood (generic function with 2 methods)

In [88]:
x, y = TV[rand(1:length(TV))]

"A águia voa pelo vale ensolarado." => "por"

In [8]:
function detector_lh(text; ngram=5, languages=LANGS, logtable=LOG_T)
    t = merged_ngrams(text, ngram)
    lhs = likelihood.(Ref(t), logtable)
    languages[argmax(lhs)]
end


detector_lh (generic function with 1 method)

In [92]:
t = merged_ngrams(x, 5)

Dict{Tuple{Vararg{UInt8}}, Float64} with 142 entries:
  (0x20, 0x65, 0x6e, 0x73, 0x6f) => 1.0
  (0x65, 0x6c, 0x6f, 0x20, 0x76) => 1.0
  (0x65, 0x6e, 0x73, 0x6f)       => 1.0
  (0x65, 0x20, 0x65, 0x6e, 0x73) => 1.0
  (0x61, 0x64, 0x6f)             => 1.0
  (0x20, 0x76)                   => 2.0
  (0x65,)                        => 3.0
  (0x6f, 0x6c, 0x61, 0x72)       => 1.0
  (0x6f, 0x20, 0x76, 0x61)       => 1.0
  (0x69,)                        => 1.0
  (0x20, 0xc3, 0xa1)             => 1.0
  (0x65, 0x20, 0x65)             => 1.0
  (0x65, 0x6c, 0x6f)             => 1.0
  (0x61, 0x64, 0x6f, 0x20)       => 1.0
  (0x20, 0x70, 0x65, 0x6c)       => 1.0
  (0x6e, 0x73, 0x6f, 0x6c)       => 1.0
  (0x75, 0x69, 0x61, 0x20, 0x76) => 1.0
  (0x6c, 0x65, 0x20, 0x65)       => 1.0
  (0x65, 0x6c, 0x6f, 0x20)       => 1.0
  ⋮                              => ⋮

In [76]:
sd = zip(LANGS, lhs)|>Dict
sd[LANGS[argmax(lhs)]], sd[y]

(-1137.6191401238673, -1140.6126625016968)

In [9]:
include("benchmark.jl")

showtable (generic function with 2 methods)

In [10]:
WV = WikiDataSet("corpus/wikipedia/test", langs=LANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [11]:
WB = benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh(t, ngram=4),
    "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=LANGS)

likelihood-n4

118.287844 seconds (1.63 G allocations: 28.844 GiB, 4.01% gc time, 0.57% compilation time)
likelihood-n3

 76.253994 seconds (885.26 M allocations: 16.353 GiB, 3.92% gc time, 0.11% compilation time)


2-element Vector{Any}:
 "likelihood-n4" => [1.0, 0.99, 1.0, 0.995, 0.995, 0.98, 0.99, 0.965, 1.0, 1.0  …  1.0, 0.99, 0.98, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
 "likelihood-n3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.98, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]

In [12]:
showtable(WB, LANGS, threshold=-0.5)

|               | Average | ara     | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|--------|--------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n

|               | Average | ara     | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|--------|--------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n4 |  97.03% | 100.00% | 99.00% | 100.00% | 99.50% | 99.50% | 98.00% | 99.00% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% | 42.50% | 100.00% | 89.50% |  99.50% | 92.00% | 93.00% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 95.00% | 98.00% | 99.00% | 99.00% | 97.00% | 92.50% |  99.50% | 99.50% | 91.00% | 100.00% | 96.00% | 100.00% | 99.00% | 98.00% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% | 99.50% |
| likelihood-n3 |  95.93% | 100.00% | 99.00% | 100.00% | 98.00% | 99.50% | 98.50% | 98.00% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.00% |  9.00% | 100.00% | 89.50% | 100.00% | 88.50% | 85.50% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.50% | 97.00% | 99.00% | 98.50% | 96.50% | 91.00% | 100.00% | 99.50% | 90.00% |  99.50% | 96.50% | 100.00% | 98.50% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% | 99.50% |


In [13]:
hbs = WikiDataSet("corpus/wikipedia/test", langs=["hbs"]);

In [14]:
x,y = hbs[rand(1:length(hbs))]
detector_lh(x, ngram=4), y

("mkd", "hbs")

In [15]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=("mkd", "hbs"))

likelihood-n5

 10.083856 seconds (90.30 M allocations: 1.666 GiB, 3.40% gc time, 1.23% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.99, 0.815]